In [3]:
import numpy as np
import pandas as pd 
from pandas import DataFrame, Series
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import random
import urllib.request
import requests
from bs4 import BeautifulSoup

In [148]:
list(range(0, 7, 2))

[0, 2, 4, 6]

In [14]:
base_url = "https://sofifa.com/players?offset="
columns = ['ID', 'Name', 'Age', 'Photo', 'Nationality', 'Flag', 'Overall', 'Potential', 'Club', 
           'Club Logo', 'Value', 'Wage', 'Special']
data_dict = {col:[] for col in columns}
data = DataFrame(columns=columns)
offset_increment = 80
for i in range(225):
    url = base_url + str(i*offset_increment)
    source_code = requests.get(url)
    plain_text = source_code.text
    soup = BeautifulSoup(plain_text)
    table_body = soup.find('tbody')
    for row in table_body.findAll('tr'):
        td = row.findAll('td')
        data_dict['Photo'].append(td[0].find('img').get('data-src'))
        data_dict['ID'].append(td[0].find('img').get('id'))
        data_dict['Nationality'].append(td[1].find('a').get('title'))
        data_dict['Flag'].append(td[1].find('img').get('data-src'))
        data_dict['Name'].append(td[1].findAll('a')[1].text)
        data_dict['Age'].append(td[2].find('div').text.strip())
        data_dict['Overall'].append(td[3].text.strip())
        data_dict['Potential'].append(td[4].text.strip())
        data_dict['Club'].append(td[5].find('a').text)
        data_dict['Club Logo'].append(td[5].find('img').get('data-src'))
        data_dict['Value'].append(td[7].text)
        data_dict['Wage'].append(td[8].text)
        data_dict['Special'].append(td[17].text)
   # data.to_csv('full_player_data.csv', encoding='utf-8')

C:\Users\Kevin\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Users\Kevin\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [63]:
def convert_currency(curr_col):
    without_euro_symbol = curr_col.str[1:]
    unit_symbol = without_euro_symbol.str[-1]
    numeric_part = np.where(unit_symbol == '0', 0, without_euro_symbol.str[:-1].pipe(pd.to_numeric))
    multipliers = unit_symbol.replace({'M':1e6, 'K':1e3}).pipe(pd.to_numeric)
    return numeric_part * multipliers

player_personal_data = (DataFrame.from_dict(data_dict)
                        .assign(EUR_value = lambda df: df['Value'].pipe(convert_currency), 
                                EUR_wage = lambda df: df['Wage'].pipe(convert_currency))
                        .drop(['Value', 'Wage'], axis=1))

In [149]:
player_personal_data

,Age,Club,Club Logo,Flag,ID,Name,Nationality,Overall,Photo,Potential,Special,EUR_value,EUR_wage
0,32,Real Madrid CF,https://cdn.sofifa.org/24/18/teams/243.png,https://cdn.sofifa.org/flags/38.png,20801,Cristiano Ronaldo,Portugal,94,https://cdn.sofifa.org/48/18/players/20801.png,94,2228,95500000.0,565000.0
1,30,FC Barcelona,https://cdn.sofifa.org/24/18/teams/241.png,https://cdn.sofifa.org/flags/52.png,158023,L. Messi,Argentina,93,https://cdn.sofifa.org/48/18/players/158023.png,93,2154,105000000.0,565000.0
2,25,Paris Saint-Germain,https://cdn.sofifa.org/24/18/teams/73.png,https://cdn.sofifa.org/flags/54.png,190871,Neymar,Brazil,92,https://cdn.sofifa.org/48/18/players/190871.png,94,2100,123000000.0,280000.0
3,30,FC Barcelona,https://cdn.sofifa.org/24/18/teams/241.png,https://cdn.sofifa.org/flags/60.png,176580,L. Suárez,Uruguay,92,https://cdn.sofifa.org/48/18/players/176580.png,92,2291,97000000.0,510000.0
4,31,FC Bayern Munich,https://cdn.sofifa.org/24/18/teams/21.png,https://cdn.sofifa.org/flags/21.png,167495,M. Neuer,Germany,92,https://cdn.sofifa.org/48/18/players/167495.png,92,1493,61000000.0,230000.0
5,28,FC Bayern Munich,https://cdn.sofifa.org/24/18/teams/21.png,https://cdn.sofifa.org/flags/37.png,188545,R. Lewandowski,Poland,91,https://cdn.sofifa.org/48/18/players/188545.png,91,2143,92000000.0,355000.0
6,26,Manchester United,https://cdn.sofifa.org/24/18/teams/11.png,https://cdn.sofifa.org/flags/45.png,193080,De Gea,Spain,90,https://cdn.sofifa.org/48/18/players/193080.png,92,1458,64500000.0,215000.0
7,26,Chelsea,https://cdn.sofifa.org/24/18/teams/5.png,https://cdn.sofifa.org/flags/7.png,183277,E. Hazard,Belgium,90,https://cdn.sofifa.org/48/18/players/183277.png,91,2096,90500000.0,295000.0
8,27,Real Madrid CF,https://cdn.sofifa.org/24/18/teams/243.png,https://cdn.sofifa.org/flags/21.png,182521,T. Kroos,Germany,90,https://cdn.sofifa.org/48/18/players/182521.png,90,2165,79000000.0,340000.0
9,29,Juventus,https://cdn.sofifa.org/24/18/teams/45.png,https://cdn.sofifa.org/flags/52.png,167664,G. Higuaín,Argentina,90,https://cdn.sofifa.org/48/18/players/167664.png,90,1961,77000000.0,275000.0


In [64]:
player_personal_data.to_csv('Complete/PlayerPersonalData.csv', index=False)

In [ ]:
player_data_url = 'https://sofifa.com/player/'
for index, row in data.iterrows():
    skill_names = []
    skill_map = {'ID' : str(row['ID'])}
    url = player_data_url + str(row['ID'])
    source_code = requests.get(url)
    plain_text = source_code.text
    soup = BeautifulSoup(plain_text)
    categories = soup.findAll('div', {'class': 'col-3'})
    for category in categories[:-1]:
        skills = category.findAll('li')
        for skill in skills:
            a = skill.text.split()
            a.reverse()
            value = a.pop()
            a.reverse()
            n = ' '.join(a)
            skill_names.append(n)
            skill_map[str(n)] = value
    master_data = DataFrame(columns=skill_names)

In [ ]:
player_data_url = 'https://sofifa.com/player/'
r = 0
for index, row in data.iterrows():
    skill_names = []
    skill_map = {'ID' : str(row['ID'])}
    url = player_data_url + str(row['ID'])
    source_code = requests.get(url)
    plain_text = source_code.text
    soup = BeautifulSoup(plain_text)
    categories = soup.findAll('div', {'class': 'col-3'})
    for category in categories[:-1]:
        skills = category.findAll('li')
        for skill in skills:
            a = skill.text.split()
            a.reverse()
            value = a.pop()
            a.reverse()
            n = ' '.join(a)
            skill_names.append(n)
            skill_map[str(n)] = value
    attr_data = DataFrame(columns=skill_names)
    for key in skill_map.keys():
        attr_data.loc[r,key] = skill_map[key]
    r = r + 1
    print(r)
    master_data = pd.concat([master_data, attr_data])
    if r % 100 == 0:
        master_data.to_csv('Complete/PlayerAttributeData.csv', encoding='utf-8')

In [140]:
player_data_url = 'https://sofifa.com/player/239736'
r = 0
skill_names = ['ID', 'Crossing', 'Finishing', 'Heading accuracy', 'Short passing', 
               'Volleys', 'Dribbling', 'Curve', 'Free kick accuracy', 'Long passing', 
               'Ball control', 'Acceleration', 'Sprint speed', 'Agility', 
               'Reactions', 'Balance', 'Shot power', 'Jumping', 'Stamina', 'Strength', 
               'Long shots', 'Aggression', 'Interceptions', 'Positioning', 
               'Vision', 'Penalties', 'Composure', 'Marking', 'Standing tackle', 
               'Sliding tackle', 'GK diving', 'GK handling', 'GK kicking', 
               'GK positioning', 'GK reflexes']
skill_dict = {name: [] for name in skill_names + ['ID']}
skill_dict['ID'].append(239736)
source_code = requests.get(player_data_url)
plain_text = source_code.text
soup = BeautifulSoup(plain_text, 'lxml')
skill_categories = soup.findAll('div', {'class': 'col-3'})

In [141]:
for category in skill_categories[:-1]: # last div is empty
    skills = category.findAll('li')
    for skill in skills:
        skill_text_list = skill.text.split()
        skill_value = int(skill_text_list[0])
        skill_name = ' '.join(skill_text_list[1:])
        skill_dict[skill_name].append(skill_value)

In [158]:
soup.find('div', class_='info')

<div class="info">
<h1>T. Trueba (ID: 239736)</h1>
<div class="meta">
<span>Tirso Trueba <a href="/players?na=83" rel="nofollow" title="Mexico"><img class="flag" data-original-set="https://cdn.sofifa.org/flags/83@2x.png 2x, https://cdn.sofifa.org/flags/83@3x.png 3x" data-src="https://cdn.sofifa.org/flags/83.png" style="width:23px;height:17px"/></a> <span class="pos pos0">GK</span>Age 21 (Jun 11, 1996) 183cm 78kg</span>
</div>
</div>

In [155]:
soup.find_all('label')

[<label class="chip" for="tag-fifa-18">FIFA 18</label>,
 <label class="chip" for="tag-fifa-17">FIFA 17</label>,
 <label class="chip" for="tag-fifa-16">FIFA 16</label>,
 <label class="chip" for="tag-fifa-15">FIFA 15</label>,
 <label class="chip" for="tag-fifa-14">FIFA 14</label>,
 <label class="chip" for="tag-fifa-13">FIFA 13</label>,
 <label class="chip" for="tag-fifa-12">FIFA 12</label>,
 <label class="chip" for="tag-fifa-11">FIFA 11</label>,
 <label class="chip" for="tag-fifa-10">FIFA 10</label>,
 <label class="chip" for="tag-fifa-09">FIFA 09</label>,
 <label class="chip" for="tag-fifa-08">FIFA 08</label>,
 <label class="chip" for="tag-fifa-07">FIFA 07</label>,
 <label>Preferred foot</label>,
 <label>International reputation</label>,
 <label>Weak foot</label>,
 <label>Skill moves</label>,
 <label>Work rate</label>,
 <label>Body type</label>,
 <label>Real face</label>,
 <label>Release clause</label>,
 <label>Position</label>,
 <label>Jersey number</label>,
 <label>Joined</label>,
 <la

In [ ]:
master_data

In [ ]:
full_data = pd.merge(data, master_data, left_index=True, right_index=True)

In [ ]:
full_data.to_csv('Allplayer.csv', encoding='utf-8')

In [ ]:
master_data.to_csv('Complete/PlayerAttributeData.csv', encoding='utf-8')

In [ ]:
full_data.to_csv('Complete/Dataset.csv', encoding='utf-8')

In [ ]:
full_data

In [ ]:
full_data.drop('Unnamed: 0', 1,  inplace=True)

In [ ]:
full_data

In [ ]:
full_data.drop('ID_x', 1,  inplace=True)

In [ ]:
full_data['ID_y']

In [ ]:
f = full_data.rename(index=str, columns={"ID_y": "ID"})

In [ ]:
f['ID']

In [ ]:
f.to_csv('Complete/Dataset.csv', encoding='utf-8')

In [ ]:
f